## We use NY Open Data portal API to get data and output them into formats we want. ##
### Fun!

**1. Vehicle registration data**

Source: Department of Motor Vehicles

In [1]:
import pandas as pd
import numpy as np
import requests

In [12]:
api_url = "https://data.ny.gov/resource/w4pv-hbkt.json?record_type=VEH?"

In [15]:
resp = requests.get(api_url)
data = resp.json()

In [16]:
data[0]

IndexError: list index out of range

In [6]:
df = pd.DataFrame(data)
df.head()

,record_type,vin,registration_class,city,state,zip,county,model_year,make,body_type,fuel_type,reg_valid_date,reg_expiration_date,scofflaw_indicator,suspension_indicator,revocation_indicator,unladen_weight,color,maximum_gross_weight
0,BOAT,999999999999,BOT,HILTON,NY,14468,MONROE,1975,STARC,BOAT,GAS,2022-08-24T00:00:00.000,2025-07-31T00:00:00.000,N,N,N,NaN,NaN,NaN
1,VEH,9999236,HIS,ROSLYN,NY,11576,NASSAU,1937,CHRY,4DSD,GAS,2023-05-09T00:00:00.000,2024-06-23T00:00:00.000,N,N,N,6300,BK,NaN
2,TRL,9999,LTR,BETHEL PARK,PA,15102,OUT-OF-STATE,1980,CRCF,LTRL,NONE,2023-11-15T00:00:00.000,2024-12-31T00:00:00.000,N,N,N,NaN,BR,3000
3,BOAT,999747,BOT,CORTLAND,NY,13045,CORTLAND,1971,STARC,BOAT,GAS,2023-03-08T00:00:00.000,2026-04-30T00:00:00.000,N,N,N,NaN,NaN,NaN
4,BOAT,999703,BOT,LOCKPORT,NY,14094,NIAGARA,1971,STARC,BOAT,NONE,2023-06-10T00:00:00.000,2026-05-31T00:00:00.000,N,N,N,NaN,NaN,NaN


In [7]:
df['reg_valid_date'].min()

'1973-04-02T00:00:00.000'

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   record_type           1000 non-null   object
 1   vin                   1000 non-null   object
 2   registration_class    1000 non-null   object
 3   city                  1000 non-null   object
 4   state                 1000 non-null   object
 5   zip                   1000 non-null   object
 6   county                1000 non-null   object
 7   model_year            1000 non-null   object
 8   make                  1000 non-null   object
 9   body_type             1000 non-null   object
 10  fuel_type             576 non-null    object
 11  reg_valid_date        1000 non-null   object
 12  reg_expiration_date   943 non-null    object
 13  scofflaw_indicator    1000 non-null   object
 14  suspension_indicator  1000 non-null   object
 15  revocation_indicator  1000 non-null   o

In [20]:
df_li = df[df['state']=='NY']

In [23]:
df_li = df[(df['county']=='NASSAU') | (df['county']=='SUFFOLK')]

In [27]:
df_li['reg_valid_date'] = pd.to_datetime(df_li['reg_valid_date'])

In [28]:
df_li['year'] = df_li['reg_valid_date'].dt.year

In [31]:
df_li = df_li[df_li['record_type']=='VEH']
df_li.head()

,record_type,vin,registration_class,city,state,zip,county,model_year,make,body_type,fuel_type,reg_valid_date,reg_expiration_date,scofflaw_indicator,suspension_indicator,revocation_indicator,unladen_weight,color,maximum_gross_weight,year
1,VEH,9999236,HIS,ROSLYN,NY,11576,NASSAU,1937,CHRY,4DSD,GAS,2023-05-09,2024-06-23T00:00:00.000,N,N,N,6300,BK,NaN,2023
49,VEH,99795N,HIS,GARDEN CITY,NY,11530,NASSAU,1948,OLDSM,CONV,GAS,2023-12-22,2025-02-01T00:00:00.000,N,N,N,4157,YW/BK,NaN,2023
136,VEH,994552057432,PAS,GLEN COVE,NY,11542,NASSAU,1955,CHEVR,2DSD,GAS,2024-03-28,2026-04-06T00:00:00.000,N,N,N,3200,OR,NaN,2024
207,VEH,9931481,PSD,MINEOLA,NY,11501,NASSAU,1993,ODB,TRLR,NONE,2014-10-09,NaN,N,N,N,27000,GR,NaN,2014
208,VEH,9931480,PSD,MINEOLA,NY,11501,NASSAU,1993,ODB,TRLR,NONE,2014-09-29,NaN,N,N,N,27000,GR,NaN,2014


In [42]:
grouped_df = df_li.groupby(['year', 'county']).size().reset_index(name='counts')

new_df = grouped_df.pivot_table(index='year', columns='county', values='counts', fill_value=0).reset_index()

In [43]:
new_df

county,year,NASSAU
0,2014,8
1,2022,3
2,2023,13
3,2024,4
